# Densely Connected Networks (DenseNet)

ResNet significantly changed the view of how to parametrize the functions in deep networks. DenseNet
is to some extent the logical extension of this. To understand how to arrive at it, let’s take a small detour
to theory. Recall the Taylor $3$ expansion for functions. For scalars it can be written as 

$$f(x) = f(0) + f'(x)x + \frac{1}{2} f''(x)x^{2} + \frac{1}{6} f(x)x^{3} + o(x^{3})$$

# Function Decomposition

The key point is that it decomposes the function into increasingly higher order terms. In a similar vein,
ResNet decomposes functions into

                                f(x) = x + g(x)

That is, ResNet decomposes f into a simple linear term and a more complex nonlinear one. What if we
want to go beyond two terms? A solution was proposed by [Huang et al, 2016](https://arxiv.org/abs/1608.06993) in the form of DenseNet, an architecture that reported record performance on the ImageNet dataset.
![](../img/densenet.svg)
Fig. 6.18: The main difference between ResNet (left) and DenseNet (right) in cross-layer connections:
use of addition and use of concatenation.

The key difference between ResNet and DenseNet is that in the latter case outputs are concatenated rather
than added. As a result we perform a mapping from x to its values after applying an increasingly complex
sequence of functions.
            $$x → [x, f 1 (x), f 2 (x, f 1 (x)), f 3 (x, f 1 (x), f 2 (x, f 1 (x)), . . .]$$
            
In the end, all these functions are combined in an MLP to reduce the number of features again. In terms of
implementation this is quite simple - rather than adding terms, we concatenate them. The name DenseNet
arises from the fact that the dependency graph between variables becomes quite dense. The last layer of
such a chain is densely connected to all previous layers. The main components that compose a DenseNet
are dense blocks and transition layers. The former defines how the inputs and outputs are concatenated,
while the latter controls the number of channels so that it is not too large.

![](../img/DenseNetDense.svg)
                           $$ Fig. 6.19: Dense connections in DenseNet $$

# Dense Blocks

DenseNet uses the modified batch normalization, activation, and convolution architecture of ResNet (see
the exercise in the previous section). First, we implement this architecture in the conv_block function.

In [1]:
import sys
sys.path.insert(0, '..')

import d2l
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

def conv_block(input_channels, num_channels):
    layers = []
    layers.append(nn.BatchNorm2d(input_channels))
    layers.append(nn.ReLU())
    layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))
    blk = nn.Sequential(*layers)
    return blk

A dense block consists of multiple conv_block units, each using the same number of output channels.
In the forward computation, however, we concatenate the input and output of each block on the channel
dimension.

In [2]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        layers = []
        for i in range(num_convs):
            layers.append(conv_block((num_channels * i + input_channels), num_channels))
        self.net = nn.Sequential(*layers)
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # Concatenate the input and output of each block on the channel
            # dimension
            X = torch.cat((X, Y), dim=1)
        return X

In the following example, we define a convolution block with two blocks of 10 output channels. When
using an input with 3 channels, we will get an output with the 3 + 2 × 10 = 23 channels. The number of
convolution block channels controls the increase in the number of output channels relative to the number
of input channels. This is also referred to as the growth rate.

In [3]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

# Transition Layers

Since each dense block will increase the number of channels, adding too many of them will lead to an
excessively complex model. A transition layer is used to control the complexity of the model. It reduces
the number of channels by using the 1 × 1 convolutional layer and halves the height and width of the
average pooling layer with a stride of 2, further reducing the complexity of the model.

In [4]:
def transition_block(input_channels, num_channels):
    layers = []
    layers.append(nn.BatchNorm2d(input_channels))
    layers.append(nn.ReLU())
    layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=1))
    layers.append(nn.AvgPool2d(kernel_size=2, stride=2))
    blk = nn.Sequential(*layers)
    return blk

Apply a transition layer with 10 channels to the output of the dense block in the previous example. This
reduces the number of output channels to 10, and halves the height and width.

In [5]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

# DenseNet Model

Next, we will construct a DenseNet model. DenseNet first uses the same single convolutional layer and
maximum pooling layer as ResNet.

In [6]:
input_channels = 64
layers = []
layers.append(nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3))
layers.append(nn.BatchNorm2d(64))
layers.append(nn.ReLU())
layers.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

Then, similar to the four residual blocks that ResNet uses, DenseNet uses four dense blocks. Similar to
ResNet, we can set the number of convolutional layers used in each dense block. Here, we set it to 4,
consistent with the ResNet-18 in the previous section. Furthermore, we set the number of channels (i.e.
growth rate) for the convolutional layers in the dense block to 32, so 128 channels will be added to each
dense block.

In ResNet, the height and width are reduced between each module by a residual block with a stride of 2.
Here, we use the transition layer to halve the height and width and halve the number of channels.

In [7]:
# Num_channels: the current number of channels
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    layers.append(DenseBlock(num_convs, input_channels, growth_rate))
    # This is the number of output channels in the previous dense block
    num_channels += num_convs * growth_rate
    # This is the number of input channels in the previous dense block
    input_channels += num_convs * growth_rate
    # A transition layer that haves the number of channels is added between
    # the dense blocks
    if i != len(num_convs_in_dense_blocks) - 1:
        layers.append(transition_block(input_channels, num_channels // 2))

Similar to ResNet, a global pooling layer and fully connected layer are connected at the end to produce
the output.

In [8]:
layers.append(nn.BatchNorm2d(input_channels))
layers.append(nn.ReLU())
layers.append(nn.AdaptiveMaxPool2d((1,1)))
layers.append(nn.Linear(input_channels, 10))

net = nn.Sequential(*layers)

# Data Acquisition and Training

Since we are using a deeper network here, in this section, we will reduce the input height and width from
224 to 96 to simplify the computation.

In [9]:
lr, num_epochs, batch_size = 0.1, 5, 256

In [19]:
from d2l.data import *

In [20]:
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)






0it [00:00, ?it/s]






  0%|          | 0/26421880 [00:00<?, ?it/s]




  0%|          | 16384/26421880 [00:00<06:24, 68763.28it/s]




  0%|          | 40960/26421880 [00:01<05:09, 85316.01it/s]




  0%|          | 49152/26421880 [00:01<05:42, 77088.19it/s]




  0%|          | 73728/26421880 [00:01<04:40, 94043.95it/s]




  0%|          | 98304/26421880 [00:01<03:53, 112837.23it/s]




  0%|          | 122880/26421880 [00:01<03:22, 129894.58it/s]




  1%|          | 139264/26421880 [00:01<03:10, 138077.30it/s]




  1%|          | 163840/26421880 [00:01<02:53, 151628.63it/s]




  1%|          | 188416/26421880 [00:01<02:38, 165256.84it/s]




  1%|          | 212992/26421880 [00:01<02:27, 177301.02it/s]




  1%|          | 237568/26421880 [00:02<02:22, 183224.91it/s]




  1%|          | 262144/26421880 [00:02<02:17, 190339.61it/s]




  1%|          | 286720/26421880 [00:02<02:13, 195627.44it/s]




  1%|          | 311296/26421880 [00:02<02:11, 198889.48it/s]




  1%|▏         | 335872/2642188

  9%|▉         | 2392064/26421880 [00:13<02:07, 189082.58it/s]




  9%|▉         | 2416640/26421880 [00:13<02:02, 195320.55it/s]




  9%|▉         | 2441216/26421880 [00:14<02:00, 198706.97it/s]




  9%|▉         | 2465792/26421880 [00:14<01:56, 205513.56it/s]




  9%|▉         | 2490368/26421880 [00:14<01:55, 207270.88it/s]




 10%|▉         | 2514944/26421880 [00:14<02:00, 199003.92it/s]




 10%|▉         | 2539520/26421880 [00:14<02:36, 152593.22it/s]




 10%|▉         | 2564096/26421880 [00:14<02:29, 160003.49it/s]




 10%|▉         | 2588672/26421880 [00:14<02:21, 168527.51it/s]




 10%|▉         | 2613248/26421880 [00:15<02:14, 177555.25it/s]




 10%|▉         | 2637824/26421880 [00:15<02:08, 184982.36it/s]




 10%|█         | 2662400/26421880 [00:15<02:04, 190544.54it/s]




 10%|█         | 2686976/26421880 [00:15<02:03, 191771.13it/s]




 10%|█         | 2711552/26421880 [00:15<02:03, 192600.10it/s]




 10%|█         | 2736128/26421880 [00:15<02:01, 194420.85it/s]

 18%|█▊        | 4808704/26421880 [00:32<14:09, 25438.67it/s] 




 18%|█▊        | 4833280/26421880 [00:32<10:21, 34761.41it/s]




 18%|█▊        | 4849664/26421880 [00:33<08:35, 41818.07it/s]




 18%|█▊        | 4882432/26421880 [00:33<06:21, 56386.45it/s]




 19%|█▊        | 4915200/26421880 [00:33<04:52, 73505.52it/s]




 19%|█▊        | 4939776/26421880 [00:33<03:53, 92162.32it/s]




 19%|█▉        | 4972544/26421880 [00:33<03:07, 114279.58it/s]




 19%|█▉        | 5013504/26421880 [00:33<02:50, 125548.85it/s]




 19%|█▉        | 5111808/26421880 [00:34<02:10, 163782.11it/s]




 19%|█▉        | 5144576/26421880 [00:34<02:24, 147249.22it/s]




 20%|█▉        | 5177344/26421880 [00:36<09:39, 36638.86it/s] 




 20%|█▉        | 5218304/26421880 [00:36<07:01, 50292.49it/s]




 20%|██        | 5292032/26421880 [00:37<05:07, 68662.16it/s]




 20%|██        | 5324800/26421880 [00:37<04:22, 80244.48it/s]




 20%|██        | 5357568/26421880 [00:37<05:09, 68158.17it/s]




 21%

 28%|██▊       | 7471104/26421880 [01:10<02:38, 119328.41it/s]




 28%|██▊       | 7487488/26421880 [01:10<03:00, 104965.18it/s]




 28%|██▊       | 7503872/26421880 [01:10<03:13, 97878.02it/s] 




 28%|██▊       | 7520256/26421880 [01:10<03:06, 101504.73it/s]




 29%|██▊       | 7536640/26421880 [01:10<02:46, 113115.22it/s]




 29%|██▊       | 7553024/26421880 [01:10<02:41, 116712.72it/s]




 29%|██▊       | 7569408/26421880 [01:11<02:53, 108672.71it/s]




 29%|██▊       | 7585792/26421880 [01:11<02:49, 111338.14it/s]




 29%|██▉       | 7602176/26421880 [01:11<02:44, 114481.15it/s]




 29%|██▉       | 7618560/26421880 [01:11<02:40, 117091.89it/s]




 29%|██▉       | 7634944/26421880 [01:11<03:13, 97266.31it/s] 




 29%|██▉       | 7651328/26421880 [01:11<03:35, 87135.23it/s]




 29%|██▉       | 7667712/26421880 [01:12<04:02, 77197.44it/s]




 29%|██▉       | 7684096/26421880 [01:12<04:07, 75701.61it/s]




 29%|██▉       | 7700480/26421880 [01:12<03:46, 82608.77it/s]





...

 96%|█████████▋| 4259840/4422102 [01:10<00:02, 66409.24it/s]





 97%|█████████▋| 4276224/4422102 [01:10<00:02, 64025.16it/s]





 97%|█████████▋| 4284416/4422102 [01:11<00:02, 63732.97it/s]





 97%|█████████▋| 4292608/4422102 [01:11<00:02, 60200.70it/s]





 97%|█████████▋| 4300800/4422102 [01:11<00:02, 56845.54it/s]





 97%|█████████▋| 4308992/4422102 [01:11<00:02, 55787.94it/s]





 98%|█████████▊| 4317184/4422102 [01:11<00:01, 53141.10it/s]





 98%|█████████▊| 4325376/4422102 [01:11<00:01, 54516.34it/s]





 98%|█████████▊| 4333568/4422102 [01:12<00:01, 45568.95it/s]





 98%|█████████▊| 4341760/4422102 [01:12<00:01, 40195.51it/s]





 98%|█████████▊| 4349952/4422102 [01:13<00:05, 13978.59it/s]





 99%|█████████▉| 4374528/4422102 [01:14<00:02, 18185.72it/s]





 99%|█████████▉| 4390912/4422102 [01:14<00:01, 24278.04it/s]





100%|█████████▉| 4415488/4422102 [01:14<00:00, 32790.69it/s]






0it [00:00, ?it/s]

Extracting /home/subham/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz









  0%|          | 0/5148 [00:00<?, ?it/s]






8192it [00:00, 8406.09it/s]             

Extracting /home/subham/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!








4423680it [01:31, 32790.69it/s]                             